In [1]:
import modin.pandas as pd
import numpy as np

In [2]:
# Quickly check annotation progress.
pd.read_parquet("wforms-ann.parquet")["status"].value_counts(dropna=False)

To remove this warning, run the following python code before doing dataframe operations:

    import ray
    ray.init()

2023-07-28 11:23:36,065	INFO worker.py:1636 -- Started a local Ray instance.


status
-1.0    11060
 1.0      388
 NaN       76
Name: count, dtype: int64

In [3]:
annos = (
    pd.read_parquet("wforms-bat.parquet")
    .join(pd.read_parquet("wforms-occ.parquet", columns=["tot"]), how="left")
    .join(pd.read_parquet("wforms-ann.parquet"), how="left")
)

# Previously attested forms and hashtags get a special status value.
annos.loc[annos["tt_att"], "status"] = +np.inf
annos.loc[annos.index.str.startswith("#"), "status"] = -np.inf

annos[annos["fst_batch"] | annos["snd_batch"]]["status"].value_counts(dropna=False)

status
 inf    14138
-1.0     7667
-inf     4619
 1.0      388
 NaN       78
Name: count, dtype: int64

In [4]:
# Normalize categories.
annos.loc[annos["category"].isna(), "category"] = None
annos["category"] = (
    annos["category"]
    .str.replace("?", "")
    .str.replace("-1", "")
    .str.replace(";", ",")
    .str.replace(" ", "")
    .str.replace("variantgrafica", "variantegrafica")
    .str.replace("enfacigrafica", "enfasigrafica")
)
annos.loc[annos["category"] == "", "category"] = None

In [5]:
# Print category usage statistics (mind the double counting from multi-category forms!)
with pd.option_context('display.max_rows', None):
    focus = annos["fst_batch"] | annos["snd_batch"]
    display(annos[focus]["category"].str.split(",").explode().value_counts())

category
forestierismo                 76
enfasigrafica                 60
univerbazione                 53
variantegrafica               49
accorciamento                 43
macedonia                     38
suffissazione                 38
prestitoadattato              30
regionalismo                  16
alterazione                   12
enfasiespressiva              11
transcategorizzazione          9
denominale                     8
deacronimico                   7
acronimo                       6
alterato                       5
deonimico                      5
ridefinizione                  4
prefissoide                    4
prefissazione                  3
suffissoide                    2
composto                       2
onomatopea                     2
metaforizzazione               2
leetspeak                      2
ilcontrariodiuniverbazione     1
siringa                        1
(admentula)regionalismo        1
dialettalismo                  1
abbreviazione                  1
d

In [6]:
with pd.option_context('display.max_rows', None):
    focus = annos["status"].isin([0,1])
    display(annos[focus].sort_values("tot", ascending=False))

,tt_att,fst_batch,snd_batch,tot,status,category,attestation,notes
wf,,,,,,,,
rdc,False,True,False,4987,1.0,acronimo,None,None
csx,False,True,False,1064,1.0,accorciamento,None,centro-sinistra
buonagiornata,False,True,False,526,1.0,univerbazione,None,None
twitteri,False,True,False,506,1.0,"deonimico,suffissazione",None,None
piddini,False,True,False,496,1.0,"suffissazione,deacronimico",None,None
tuitteri,False,True,False,393,1.0,"deonimico,suffissazione",None,None
pidioti,False,True,False,377,1.0,macedonia,None,idiota del partito democratico
yeahhhhhh,False,True,False,365,1.0,enfasigrafica,None,None
reel,False,False,True,315,1.0,forestierismo,None,None
